<a href="https://colab.research.google.com/github/shawwn/colab-tricks/blob/master/ngrok-tricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount your google drive & symlink it to `/drive`

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#(optional) If a TPU is attached, print its address

This notebook works for all runtime types: CPU, GPU, and TPU.

If connected to TPU runtime, this cell prints info about the attached TPU. (This is just for convenience / informational purposes.)

In [0]:
import os
import pprint
%tensorflow_version 2.x
import tensorflow as tf

if 'COLAB_TPU_ADDR' in os.environ:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  assert os.environ['COLAB_TPU_ADDR'] == tpu.cluster_spec().as_dict()['worker'][0]
  tf.config.experimental_connect_to_cluster(tpu)
  tf.distribute.experimental.TPUStrategy(tpu)
else:
  gpu=tf.config.get_visible_devices('GPU')
  if gpu:
    print('Running on GPU', gpu)
  else:
    print('Not connected to TPU/GPU runtime')

TPU address is grpc://10.44.120.90:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1173016086274077236),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2991867235668810802),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6184859761337604570),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2958198024988600890),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7018531718559331614),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4577201967883118425),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1807161912338548618),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 6466936639391228903),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 5609808055865915

# Install `sshd`, `tmux`, and `vim`

You can add any other prerequisites you want, e.g. `emacs`. That way, when you ssh into the colab instance, all of these will already be installed.

(You'd otherwise have to run `apt-get install emacs` every time you ssh into a fresh colab instance. It's easier just to install all your favorite programs here, ahead of time.)

In [0]:
!apt-get install ssh tmux vim

# Create an sshd configuration file

- Normally, you shouldn't need to modify this next cell; the configuration is designed to work with colab.

Simply run this cell:

In [0]:
#@title
%%writefile /drive/ngrok-ssh/sshd_config
#	$OpenBSD: sshd_config,v 1.101 2017/03/14 07:19:07 djm Exp $

# This is the sshd server system-wide configuration file.  See
# sshd_config(5) for more information.

# This sshd was compiled with PATH=/usr/bin:/bin:/usr/sbin:/sbin

# The strategy used for options in the default sshd_config shipped with
# OpenSSH is to specify options with their default value where
# possible, but leave them commented.  Uncommented options override the
# default value.

#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

#HostKey /etc/ssh/ssh_host_rsa_key
#HostKey /etc/ssh/ssh_host_ecdsa_key
#HostKey /etc/ssh/ssh_host_ed25519_key

# Ciphers and keying
#RekeyLimit default none

# Logging
#SyslogFacility AUTH
#LogLevel INFO

# Authentication:

#LoginGraceTime 2m
#PermitRootLogin prohibit-password
#StrictModes yes
#MaxAuthTries 6
#MaxSessions 10

#PubkeyAuthentication yes

# Expect .ssh/authorized_keys2 to be disregarded by default in future.
#AuthorizedKeysFile	.ssh/authorized_keys .ssh/authorized_keys2

#AuthorizedPrincipalsFile none

#AuthorizedKeysCommand none
#AuthorizedKeysCommandUser nobody

# For this to work you will also need host keys in /etc/ssh/ssh_known_hosts
#HostbasedAuthentication no
# Change to yes if you don't trust ~/.ssh/known_hosts for
# HostbasedAuthentication
#IgnoreUserKnownHosts no
# Don't read the user's ~/.rhosts and ~/.shosts files
#IgnoreRhosts yes

# To disable tunneled clear text passwords, change to no here!
#PasswordAuthentication yes
#PermitEmptyPasswords no

# Change to yes to enable challenge-response passwords (beware issues with
# some PAM modules and threads)
ChallengeResponseAuthentication no

# Kerberos options
#KerberosAuthentication no
#KerberosOrLocalPasswd yes
#KerberosTicketCleanup yes
#KerberosGetAFSToken no

# GSSAPI options
#GSSAPIAuthentication no
#GSSAPICleanupCredentials yes
#GSSAPIStrictAcceptorCheck yes
#GSSAPIKeyExchange no

# Set this to 'yes' to enable PAM authentication, account processing,
# and session processing. If this is enabled, PAM authentication will
# be allowed through the ChallengeResponseAuthentication and
# PasswordAuthentication.  Depending on your PAM configuration,
# PAM authentication via ChallengeResponseAuthentication may bypass
# the setting of "PermitRootLogin without-password".
# If you just want the PAM account and session checks to run without
# PAM authentication, then enable this but set PasswordAuthentication
# and ChallengeResponseAuthentication to 'no'.
UsePAM yes

#AllowAgentForwarding yes
AllowTcpForwarding yes
#GatewayPorts no
X11Forwarding yes
#X11DisplayOffset 10
#X11UseLocalhost yes
#PermitTTY yes
PrintMotd no
#PrintLastLog yes
#TCPKeepAlive yes
#UseLogin no
PermitUserEnvironment yes
#Compression delayed
#ClientAliveInterval 0
#ClientAliveCountMax 3
#UseDNS no
#PidFile /var/run/sshd.pid
#MaxStartups 10:30:100
#PermitTunnel no
#ChrootDirectory none
#VersionAddendum none

# no default banner path
#Banner none

# Allow client to pass locale environment variables
AcceptEnv LANG LC_*

# override default of no subsystems
Subsystem	sftp	/usr/lib/openssh/sftp-server

# Example of overriding settings on a per-user basis
#Match User anoncvs
#	X11Forwarding no
#	AllowTcpForwarding no
#	PermitTTY no
#	ForceCommand cvs server

Writing /drive/ngrok-ssh/sshd_config


In [ ]:
!env | sed -E "s/=(.*[<>].*)/=\'\1\'/" >> ~/.ssh/environment

# Paste the contents of your local `~/.ssh/id_rsa.pub` file into this next cell

- If you don't have an `~/.ssh/id_rsa.pub` file, use `ssh-keygen` to make one.

- Remember to actually run the cell after you paste your key into it!



In [0]:
%%writefile ~/.ssh/authorized_keys
REPLACE_THIS_LINE_WITH_YOUR_ID_RSA_PUB_FILE

Overwriting /root/.ssh/authorized_keys


# Restart sshd

In [0]:
!cp /drive/ngrok-ssh/sshd_config /etc/ssh/sshd_config
!service ssh restart
!mkdir -p ~/.ssh

 * Restarting OpenBSD Secure Shell server sshd
   ...done.


#Install ngrok

In [0]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2019-11-13 11:34:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.86.66.223, 34.197.213.11, 52.204.38.150, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.86.66.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  30.6MB/s    in 0.4s    

2019-11-13 11:34:55 (30.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip


# Sign-in with ngrok

- Sign up for a free ngrok account: https://ngrok.com

- Copy your authtoken from https://dashboard.ngrok.com/auth and paste it into the following cell

- Run the cell

In [0]:
!/ngrok authtoken PASTE_YOUR_AUTHTOKEN_HERE

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Create various ngrok tunnel configurations

Run each of these cells, one by one:

In [0]:
%%writefile /drive/ngrok-ssh/ssh.yml
tunnels:
  ssh:
    proto: tcp
    addr: 22

Writing /drive/ngrok-ssh/ssh.yml


In [0]:
%%writefile /drive/ngrok-ssh/tensorboard.yml
tunnels:
  tensorboard:
    proto: http
    addr: 6006
    inspect: false
    bind_tls: true

Overwriting /drive/ngrok-ssh/tensorboard.yml


In [0]:
%%writefile /drive/ngrok-ssh/http8080.yml
tunnels:
  http8080:
    proto: http
    addr: 8080
    inspect: false
    bind_tls: true

Overwriting /drive/ngrok-ssh/http8080.yml


In [0]:
%%writefile /drive/ngrok-ssh/tcp8080.yml
tunnels:
  tcp8080:
    proto: tcp
    addr: 8080

Writing /drive/ngrok-ssh/tcp8080.yml


In [0]:
import os
if 'COLAB_TPU_ADDR' in os.environ:
  with open('/drive/ngrok-ssh/tpu.yml', 'w') as f:
    f.write("""
tunnels:
  tpu:
    proto: tcp
    addr: {}
""".format(os.environ['COLAB_TPU_ADDR']).strip())
  print('Wrote /drive/ngrok-ssh/tpu.yml')
!touch /drive/ngrok-ssh/tpu.yml # File existence is required for following commands

Wrote /drive/ngrok-ssh/tpu.yml


In [0]:
%%writefile /drive/ngrok-ssh/run_ngrok.sh
#!/bin/sh
set -x
/ngrok start --config ~/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout --config /drive/ngrok-ssh/tensorboard.yml --config /drive/ngrok-ssh/http8080.yml --config /drive/ngrok-ssh/tcp8080.yml --config /drive/ngrok-ssh/tpu.yml "$@"

Overwriting /drive/ngrok-ssh/run_ngrok.sh


#Start ngrok

In [0]:
import os
if 'COLAB_TPU_ADDR' in os.environ:
  get_ipython().system_raw('bash /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 tpu &')
else:
  get_ipython().system_raw('bash /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 &')

In [0]:
!ps aux | grep ngrok

root        2345  0.0  0.0  39192  6660 ?        S    10:32   0:00 bash /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 tpu
root        2346  0.2  0.1 122768 19936 ?        Sl   10:32   0:01 /ngrok start --config /root/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout --config /drive/ngrok-ssh/tensorboard.yml --config /drive/ngrok-ssh/http8080.yml --config /drive/ngrok-ssh/tcp8080.yml --config /drive/ngrok-ssh/tpu.yml ssh tensorboard tcp8080 tpu
root        2599  0.0  0.0  39196  6620 ?        S    10:43   0:00 /bin/bash -c ps aux | grep ngrok
root        2601  0.0  0.0  38568  4876 ?        S    10:43   0:00 grep ngrok


# Make an ngrok API
Let's define some helper functions:

- `get_ngrok_info()` - returns a JSON object containing info about all open ngrok tunnels

- `get_ngrok_tunnels()` - enumerate over all open ngrok tunnels. Yields `name, info` tuples.

- `get_ngrok_tunnel(name)` - get info about a tunnel

- `get_ngrok_url(name, local=False)` - get a tunnel's URL. If `local` is `False`, returns a public URL that anyone can access, otherwise returns the local url being forwarded

In [0]:
import requests
import urllib.parse

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']

# Skip this section

These are various commands for debugging ngrok.

In [0]:
#!bash /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 tpu

+ /ngrok start --config /root/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout --config /drive/ngrok-ssh/tensorboard.yml --config /drive/ngrok-ssh/http8080.yml --config /drive/ngrok-ssh/tcp8080.yml --config /drive/ngrok-ssh/tpu.yml ssh tensorboard tcp8080 tpu
INFO[11-13|10:31:21] open config file                         path=/root/.ngrok2/ngrok.yml err=nil
INFO[11-13|10:31:21] open config file                         path=/drive/ngrok-ssh/ssh.yml err=nil
INFO[11-13|10:31:21] open config file                         path=/drive/ngrok-ssh/tensorboard.yml err=nil
INFO[11-13|10:31:21] open config file                         path=/drive/ngrok-ssh/http8080.yml err=nil
INFO[11-13|10:31:21] open config file                         path=/drive/ngrok-ssh/tcp8080.yml err=nil
INFO[11-13|10:31:21] open config file                         path=/drive/ngrok-ssh/tpu.yml err=nil
t=2019-11-13T10:31:21+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
t=2019-11-13T10:3

In [0]:
#!curl -s http://localhost:4040/api/tunnels  | python3 -c "import sys, json; url=json.load(sys.stdin)['tunnels'][0]['public_url']; import urllib.parse; print(urllib.parse.urlparse(url).netloc)"

0.tcp.ngrok.io:10596


#Start Tensorboard

In [0]:
LOG_DIR = '/drive/gpt-2-train/checkpoint/run1'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [0]:
!ps aux | grep tensorboard

root        1588  0.2  2.2 872976 304868 ?       Sl   10:00   0:06 /usr/bin/python2 /usr/local/bin/tensorboard --logdir /drive/gpt-2-train/checkpoint/run1 --host 0.0.0.0 --port 6006
root        2345  0.0  0.0  39192  6660 ?        S    10:32   0:00 bash /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 tpu
root        2346  0.2  0.1 122768 19348 ?        Sl   10:32   0:01 /ngrok start --config /root/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout --config /drive/ngrok-ssh/tensorboard.yml --config /drive/ngrok-ssh/http8080.yml --config /drive/ngrok-ssh/tcp8080.yml --config /drive/ngrok-ssh/tpu.yml ssh tensorboard tcp8080 tpu
root        2500  0.0  0.0  39192  6624 ?        S    10:39   0:00 /bin/bash -c ps aux | grep tensorboard
root        2502  0.0  0.0  38572  5684 ?        S    10:39   0:00 grep tensorboard


# How to view tensorboard

In [0]:
print('To view tensorboard, visit', get_ngrok_url('tensorboard'))

To view tensorboard, visit https://1f6310e7.ngrok.io


# Display each ngrok URL

For each ngrok tunnel, this cell prints its name, its public URL, and the local URL being forwarded to that public URL.

In [0]:
for name, tunnel in get_ngrok_tunnels():
  local_url = get_ngrok_url(name, local=True)
  public_url = get_ngrok_url(name, local=False)
  print('{:12s} {} <-> {}'.format(name, public_url, local_url))

tpu          tcp://0.tcp.ngrok.io:16601 <-> 10.44.120.90:8470
ssh          tcp://0.tcp.ngrok.io:10596 <-> localhost:22
tcp8080      tcp://0.tcp.ngrok.io:10056 <-> localhost:8080
tensorboard  https://1f6310e7.ngrok.io <-> http://localhost:6006


# How to SSH into this Colab instance

In [0]:
print('')
public_url, public_port = urllib.parse.urlparse(get_ngrok_url('ssh')).netloc.split(':')
print('To SSH into this colab instance, run the following command on your local machine:')
print('')

print('  ssh -p {} root@{}'.format(public_port, public_url))


To SSH into this colab instance, run the following command on your local machine:

  ssh -p 10596 root@0.tcp.ngrok.io
